## Glare Estimator

Tool takes in as input the longitude, latitude, camera bearing, camera depression angle, focal length, sensor width and sensor height.

Outputs a glare graph showing times of glare within the field of view for all days of the year. Glare graph is saved as an image and datapoints are also saved in a csv file.

The below 2 images show important parameters from the paper "A Globally Verified Coastal Glare Estimation Tool".

There is also a smartphone verification cell at the bottom of the notebook to recreate the results in section 3.1.

To run this program successfully, you will need to install astropy https://docs.astropy.org/en/stable/install.html#anaconda-install and opencv https://anaconda.org/conda-forge/opencv

You will also need the following in your working directory:
- th_dist_cm.csv
- phone_images/ - this folder is only necessary for running the last cell for verification with phone images, the main functionality (first two cells) does not require this folder

![title](glare_figure_4.jpg)

![title](glare_figure_5.jpg)

Choose what type of glare graph calculations to use and input variables below, then run the cell below to do imports and load function. The Palm Beach and Tweed Heads camera parameters are shown as examples.

In [ ]:
# imports
from datetime import datetime, timedelta
import numpy as np
import csv
import time
import math
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors
import astropy.coordinates as coord
from astropy.time import Time
import astropy.units as u
import cv2
import os

"""*****************    Set your camera and location parameters here    *******************"""
""""""
# for Palm Beach QLD
cam_focal_length = 19.27 # mm
cam_sensor_width = 6.058 # mm
cam_sensor_height = 4.415 # mm
longitude = 153.4665
latitude = -28.1108
th_cam_h = 47.5 # degrees
th_cam_v = 5 # degrees
year_to_calc = 2021
utc_time_add = 10 # e.g. +10 for QLD

# these lists are only for verification points
#                              J             F              M               A            M               J              J                A              S                  O               N      
glare_check_days =          [   15,    22,  7+31, 22+31,  5+59,  24+59,  9+90,  23+90, 8+120, 28+120, 6+151, 21+151, 5+181, 21+181, 7+212, 22+212,  7+243,  22+243,   7+273,  24+273, 13+304, 20+304, 9+334]
glare_start_check_seconds = [31800, 37800, 32940, 32940, 29340,  26340, 28140,  28140, 26340,  26340, 25740,  25740, 25740,  26100, 28740,  28740,  28140,   26400,   28140,   28140,  29940,  29940, 30540]
glare_finish_check_seconds= [41400, 40800, 40140, 43140, 38940,  41340, 39540,  39600, 36540,  34740, 35340,  39540, 35340,  39300, 35940,  40740,  39540,   40260,   40740,   37740,  35340,  38280, 39540]
""""""
"""
# for Tweed River, NSW/QLD
cam_focal_length = 11.515 # mm
cam_sensor_width = 6.058 # mm
cam_sensor_height = 4.415 # mm
longitude = 153.5489
latitude = -28.1654
th_cam_h = 110 # degrees
th_cam_v = 1.5 # degrees
year_to_calc = 2022
utc_time_add = 10 # +10 since camera is actually on the QLD side of the border

# these lists are only for verification points
#                              J             F              M               A            M               J              J                A              S                  O               N      
glare_check_days =         [   15,    22,  7+31, 22+31,  5+59,                                                                                                      6+273, 20+273,  13+304, 20+304,  6+334,  22+334]
glare_start_check_seconds =[18660, 18660, 19860, 19860, 20460,                                                                                                      19260,  18060,   17460,  16860,  16860,   17460]
glare_finish_check_seconds=[32460, 32460, 34860, 30060, 32460,                                                                                                      31860,  31260,   31260,  33660,  36660,   33660]
"""

# glare physics variables - best combination from paper was found to be "wave" and "True".
th_dist_type = "wave" # flat distribution, or wave distribution
schlick_flag = True # True for using schlicks approximation, False for not

# glare graph resolution - for a quick graph, set monthly_flag to True
# 0: standard monthly glare graph. Run at 10 minute intervals for the 15th of each month. Fastest run type (less than a minute).
# 1: standard full glare graph. Run at 1 minute intervals for every day of the year. Slowest run type (about 20 minutes)
# 2: verification full glare graph. Used to create Figure 13a,b in the paper. (about 20 minutes)
# alternatively you can comment out the run_type section and make your own custom settings
run_type = 0
if run_type == 0:
    ver_flag = False
    monthly_flag = True
    min_interval = 10
    month_limit = 12
elif run_type == 1:
    ver_flag = False
    monthly_flag = False
    min_interval = 1
    month_limit = 12
elif run_type == 2:
    ver_flag = True
    monthly_flag = False
    min_interval = 1
    month_limit = 12
else:
    print("run_type value is not acceptable")

"""############## 1: Setup 2D matrix of θ_wx,θ_wy values within ±30°, with a polar limit mask #############"""
# Angle ranges = if you want to change these you need to make a new th_dist_cm.csv file with correct size 
th_wx_max = 30 # ± degrees
th_wy_max = th_wx_max # ± degrees
th_dist = np.loadtxt(open("th_dist_cm.csv", "rb"), delimiter=",", skiprows=0)

if th_dist_type == "flat":
    # try flat distribution
    th_dist_f = np.ones_like(th_dist)
elif th_dist_type == "wave":
    # import theta distribution of roughness of surface from csv file
    th_dist_f = th_dist
else:
    print("th_dist not specified, therefore choosing flat distribution by default")
    th_dist_f = np.ones_like(th_dist)

th_dist_f = th_dist_f.flatten(order='C')

# Setup 2D grid of θ_wx,θ_wy values within ±30° polar
th_wx, th_wy = np.meshgrid(np.linspace(-th_wx_max, th_wx_max, 2*th_wx_max + 1), np.linspace(-th_wy_max, th_wy_max, 2*th_wy_max + 1))
th_w_mask = np.ones_like(th_wx)
for i in range(0, len(th_wx), 1):
    for j in range(0, len(th_wx[0]), 1):
        th_polar = np.rad2deg(np.arccos( 1/np.sqrt(((np.tan(np.deg2rad(th_wx[i][j])))**2) + (np.tan(np.deg2rad(th_wy[i][j]))**2) + 1) ))
        if th_polar > th_wx_max:
            th_w_mask[i][j] = 0

# apply mask so that angles greater than th_wx_max aren't included
th_dist_f = np.multiply(th_dist_f, th_w_mask.flatten(order='C'))

if min_interval > 30 or (int(60/min_interval) != 60/min_interval) or min_interval < 1:
    print("minute interval must be less than 30, greater than or equal to 1 and a multiple of 60. You need to re run this cell with an acceptable interval before running the other cells")
else:
    pass

def calc_day_glare(ty, tm, td, ts, loc, cam_view_horiz, cam_view_vert, 
                   th_wx_max, th_wy_max, th_cam_h_min, th_cam_h_max, th_cam_v_min, th_cam_v_max, 
                   th_dist_f, schlick_flag, th_wx, th_wy, th_w_mask, min_interval, utc_time_add):
    closest_zen_rise = 0 # far out initial value which will change
    closest_zen_set = 0 # far out initial value which will change
    glare_perc_list = []

    """############ 3: Extract θ_zen,ϕ_az from Astropy using known latitude, longitude, time and timezone ##############"""
    time_vals = []
    time_vals_nostr = []
    for j in range(0, 24, 1):
        th = j
        # check all minute intervals
        for k in range(0, 60, min_interval):
            tmin = k
            time_vals.append(str(datetime(ty, tm, td, th, tmin, ts) - timedelta(hours = utc_time_add)).replace(" ", "T"))
            time_vals_nostr.append(datetime(ty, tm, td, th, tmin, ts))

    input_time = Time(time_vals, format='isot', scale='utc') # UTC time
    altaz = coord.AltAz(location=loc, obstime=input_time)
    sun = coord.get_sun(input_time)

    # convert azimuth and zenith to degrees
    az_list = sun.transform_to(altaz).az.degree
    zen_list = sun.transform_to(altaz).zen.degree

    # check all hours
    for j in range(0, 24, 1):
        glare_times_j = []

        # check all minute intervals
        for k in range(0, 60, min_interval):
            th_az = az_list[int((j)*(60/min_interval) + k/min_interval)]
            th_zen = zen_list[int((j)*(60/min_interval) + k/min_interval)]
            time_val = time_vals_nostr[int((j)*(60/min_interval) + k/min_interval)]

            # get sunrise and sunset
            # keep closest value to 90 deg zenith with 0>azimuth>180 for sunrise (rises from the east)
            if th_az >= 0 and th_az <=180:
                # check if closer to 90 deg than last value
                if abs(90 - closest_zen_rise) > abs(90 - th_zen):
                    sunrise_time = (time_val.hour * 60 + time_val.minute) * 60 + time_val.second # get the epoch of the time value  # time_val.time()
                    closest_zen_rise = th_zen
            # keep closest value to 90 deg zenith with 180>azimuth>360 for sunset (sets in the west)
            if th_az > 180 and th_az <=360:
                # check if closer to 90 deg than last value
                if abs(90 - closest_zen_set) > abs(90 - th_zen):
                    sunset_time = (time_val.hour * 60 + time_val.minute) * 60 + time_val.second # get the epoch of the time value  # time_val.time()
                    closest_zen_set = th_zen

            """############ 4: If daytime (θ_zen ≤ 90): ##############"""
            # if the sun is not up, assume there is no glare and skip to next iteration
            if th_zen > 90:
                glare_perc_list.append(0)
                continue

            """############ 5: Calc. ϕ_(ref,h) and θ_(ref,v) for pairs of θ_wx,θ_wy in matrix (Eqs. 5 and 6, Eqs. 9 and 10 for θ_zen < 2θ_wx) ###########"""
            th_sv = 90 - th_zen

            # Schlicks's approximation
            if schlick_flag == True:
                # calculate angle of incidence between sunray and surface normal
                th_wx_rd = np.deg2rad(th_wx)
                th_wy_rd = np.deg2rad(th_wy)
                th_zen_rd = np.deg2rad(th_zen)
                th_az_rd = np.deg2rad(0) # this is always oriented in the direction of the ray of light (always 0 for the way the geometry is setup)
                u_vec = np.array([np.tan(th_wx_rd), np.tan(th_wy_rd), 1])
                v_vec = np.array([np.sin(th_zen_rd)*np.cos(th_az_rd), np.sin(th_zen_rd)*np.sin(th_az_rd), np.cos(th_zen_rd)])
                th_i_num = u_vec.dot(v_vec)
                th_i_den = np.sqrt(u_vec.dot(u_vec))*np.sqrt(v_vec.dot(v_vec))
                th_i = np.rad2deg(np.arccos(th_i_num/th_i_den))
                np.where(np.isnan(th_i), 0, th_i)
                r_coef = 0.0209 + 0.9791*(1 - np.cos(np.deg2rad(th_i)))**5

            # handle out of bounds angles
            th_wx_1 = np.where(th_zen < 2*th_wx, th_wx, np.NAN)
            th_wy_1 = np.where(th_zen < 2*th_wx, th_wy, np.NAN)
            th_wx_2 = np.where(th_zen >= 2*th_wx, th_wx, np.NAN)
            th_wy_2 = np.where(th_zen >= 2*th_wx, th_wy, np.NAN)

            # Equation 8,10 for special flip direction case
            num = np.multiply(np.tan(np.deg2rad(180 - (th_sv + 2*th_wx_1))), np.cos(np.deg2rad(2*th_wy_1)))
            den = np.sqrt(1 + np.multiply(np.square(np.tan(np.deg2rad(180 - (th_sv + 2*th_wx_1)))), np.square(np.sin(np.deg2rad(2*th_wy_1)))))
            th_wv = np.rad2deg(np.arctan(np.divide(num, den)))
            th_ref_v_1 = th_wv

            # Equation 7,9 for special flip direction case
            th_az_dev = -1*np.rad2deg(np.arctan(np.multiply(np.tan(np.deg2rad(180 - (th_sv + 2*th_wx_1))), np.sin(np.deg2rad(2*th_wy_1)))))
            th_ref_h_1 = ((th_az + 180) % 360) + th_az_dev

            # Equation 4,6 for standard case
            num = np.multiply(np.tan(np.deg2rad(th_sv + 2*th_wx_2)), np.cos(np.deg2rad(2*th_wy_2)))
            den = np.sqrt(1 + np.multiply(np.square(np.tan(np.deg2rad(th_sv + 2*th_wx_2))), np.square(np.sin(np.deg2rad(2*th_wy_2)))))
            th_wv = np.rad2deg(np.arctan(np.divide(num, den)))
            th_ref_v_2 = th_wv

            # Equation 3,5 for standard case
            th_az_dev = np.rad2deg(np.arctan(np.multiply(np.tan(np.deg2rad(th_sv + 2*th_wx_2)), np.sin(np.deg2rad(2*th_wy_2)))))
            th_ref_h_2 = th_az + th_az_dev

            # combine matrices again with correct indexing
            th_ref_v_1[np.isnan(th_ref_v_1)] = 0
            th_ref_h_1[np.isnan(th_ref_h_1)] = 0
            th_ref_v_2[np.isnan(th_ref_v_2)] = 0
            th_ref_h_2[np.isnan(th_ref_h_2)] = 0
            th_ref_v = th_ref_v_1 + th_ref_v_2
            th_ref_h = th_ref_h_1 + th_ref_h_2           

            """############ 6: For every ≈ 2.1°×2.1° cell in the FOV: ############"""
            # setup the range of cells at about 2.1 degrees per cell
            v_inc = (th_cam_v_max - th_cam_v_min)/2.1
            h_inc = (th_cam_h_max - th_cam_h_min)/2.1
            th_cam_vs = np.linspace(th_cam_v_min, th_cam_v_max, int(v_inc))
            th_cam_hs = np.linspace(th_cam_h_min, th_cam_h_max, int(h_inc))
            all_deps = th_ref_v.flatten(order='C')
            all_azs = th_ref_h.flatten(order='C')
            glare_percs_i = []
            for m in range(0, len(th_cam_vs), 1):
                for n in range(0, len(th_cam_hs), 1):
                    """############ 7: Set Φ_(ref,h) values to 1 if within range of ϕ_(cam,h), set Θ_(ref,v) values to 1 if within range of θ_(cam,v) #############"""
                    # set to 0 all th_wx and th_wy that don't reflect into the camera fov
                    th_cam_v_min_m = th_cam_vs[m] - 0.5*(th_cam_v_max - th_cam_v_min)/(int(v_inc))
                    th_cam_v_max_m = th_cam_vs[m] + 0.5*(th_cam_v_max - th_cam_v_min)/(int(v_inc))
                    th_cam_h_min_n = th_cam_hs[n] - 0.5*(th_cam_h_max - th_cam_h_min)/(int(h_inc))
                    th_cam_h_max_n = th_cam_hs[n] + 0.5*(th_cam_h_max - th_cam_h_min)/(int(h_inc))

                    # if the camera has any FOV above the horizon, don't include any negative th_ref_v angles, 
                    # rather there should be 0 reflection since there is no water here. These regions of the image 
                    # are not included in the glare percentage. The glare percentage is the portion of glare below
                    # the horizon on the water.
                    if th_cam_v_max_m < 0:
                        continue
            
                    # work out glare percentage
                    deps_glare_1 = np.where(all_deps >= th_cam_v_min_m, 1, 0)
                    deps_glare_2 = np.where(all_deps <= th_cam_v_max_m, 1, 0)
                    deps_glare = deps_glare_1 & deps_glare_2
                    azs_glare_1 = np.where(all_azs >= th_cam_h_min_n, 1, 0)
                    azs_glare_2 = np.where(all_azs <= th_cam_h_max_n, 1, 0)
                    azs_glare = azs_glare_1 & azs_glare_2
                    
                    """############ 8: Θ_all=(Φ_(ref,h)  AND Θ_(ref,v) )*Θ_dist*Θ_R(θ_i ) ##########"""
                    # apply roughness distribution
                    th_all = np.multiply(deps_glare & azs_glare, th_dist_f)

                    # apply Schlicks's approximation
                    if schlick_flag == True:
                        th_all = np.multiply(th_all, r_coef.flatten(order='C'))
                    
                    """############ 9: glare_percentage_i = (∑Θ_all)/(∑Θ_dist )*100 ##########"""
                    # get volume under roughness distribution 
                    #- since width and length of each cell is the same for both distributions, 
                    # they cancel when used for a ratio. Therefore only need to sum the bin heights.
                    vol_dist = np.sum(th_dist_f)
                    
                    # get volume under output distribution
                    vol_output = np.sum(th_all)
                    
                    # find percentage of total volume
                    glare_perc = (vol_output/vol_dist)*100

                    # add to glare percentage average and the image
                    glare_percs_i.append(glare_perc)
            
            """############ 10: glare percentage = mean(glare_percentage_i’s) ############"""
            glare_perc_average = np.mean(glare_percs_i)
            glare_perc_list.append(glare_perc_average)

        # debugging line
        #print("month " + str(i) + ", day " + str(td) + " and hour " + str(j) + " sun az = " + str(round(th_az,2)) + " sun zen = " + str(round(th_zen,2)))

    return sunrise_time, sunset_time, glare_perc_list

print("done")

Run the below cell to create the glare graph

In [ ]:
filename = "All_sun_calcs_for_" + str(year_to_calc) + "y" + str(round(latitude,4)) + \
"lat" + str(round(longitude, 4)) + "long_dist_type_" + th_dist_type + "_schlick_" + str(schlick_flag) + "_ver"

# Algorithm input parameters setup
cam_view_horiz = 2*math.degrees(math.atan(cam_sensor_width/(2*cam_focal_length)))  # angle range for view
cam_view_vert = 2*math.degrees(math.atan(cam_sensor_height/(2*cam_focal_length)))  # angle range for view 
th_cam_h_max = (th_cam_h + cam_view_horiz/2)
th_cam_h_min = (th_cam_h - cam_view_horiz/2)
th_cam_v_max = th_cam_v + cam_view_vert/2
th_cam_v_min = th_cam_v - cam_view_vert/2 
print("Horizontal field of view = " + str(round(cam_view_horiz, 2)))
print("Vertical field of view = " + str(round(cam_view_vert,2)))

#https://stackoverflow.com/questions/47663082/how-can-we-compute-solar-position-at-a-given-place-on-a-given-day-and-time
loc = coord.EarthLocation(lon = longitude*u.deg, lat=latitude*u.deg)
ty = year_to_calc
#             #J   F   M    A   M   J   J   A   S   O   N   D
td_las_list = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
ts = 0

# for day 1
sunset_times = []
sunrise_times = []
glare_percs = []

"""############# 2: For every minute, of every hour, day, month: ##############"""
for i in range(1, month_limit + 1, 1):
    tm = i
    ty = year_to_calc
    # handle leap years
    if i == 1:
        if ty % 4 == 0:
            if ty % 100 == 0:
                if ty % 400 == 0:
                    td_las_list[1] = 29
                else:
                    pass
            else:
                td_las_list[1] = 29

    if monthly_flag == True:
        td = 15
        sunrise_time, sunset_time, glare_percs_list = calc_day_glare(ty, tm, td, ts, loc, cam_view_horiz, cam_view_vert, th_wx_max, th_wy_max, th_cam_h_min, th_cam_h_max, th_cam_v_min, th_cam_v_max, th_dist_f, schlick_flag, th_wx, th_wy, th_w_mask, min_interval, utc_time_add)
        sunset_times.append(sunset_time)
        sunrise_times.append(sunrise_time)
        glare_percs.append(glare_percs_list)
    else:
        # for every day
        for j in range(1, td_las_list[i-1]+1, 1):
            td = j
            print("Processing month " + str(tm) + " and day " + str(td))
            sunrise_time, sunset_time, glare_percs_list = calc_day_glare(ty, tm, td, ts, loc, cam_view_horiz, cam_view_vert, th_wx_max, th_wy_max, th_cam_h_min, th_cam_h_max, th_cam_v_min, th_cam_v_max, th_dist_f, schlick_flag, th_wx, th_wy, th_w_mask, min_interval, utc_time_add)
            sunset_times.append(sunset_time)
            sunrise_times.append(sunrise_time)
            glare_percs.append(glare_percs_list)

"""################# 11: Plot glare percentage value for all times of the day and all days of the year in a graph ##############"""
%matplotlib qt
months_axis = list(range(1, month_limit+1, 1)) # 12 months in a year
days_axis = list(range(1, len(sunrise_times)+1, 1))
glare_volumes = np.array(glare_percs)

if monthly_flag == True:
    # produce a csv file with times
    open(filename + ".csv", "wb")
    with open(filename + ".csv", "a", newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Day", "Sunrise", "Sunset", "Glare Index"])
        for i in range(0, len(months_axis), 1):
            sunrise_time = str(timedelta(seconds=sunrise_times[i]))
            sunset_time = str(timedelta(seconds=sunset_times[i]))
            row = [months_axis[i], sunrise_time, sunset_time]
            row.extend(list(glare_volumes[i]))
            writer.writerow(row)

    # plot sunsets and sunrises and glare times
    fig = plt.figure(figsize=(9.004, 3.869), dpi=100)
    ax=fig.add_subplot(111)
    ax.set_xlabel('Month of Year (on 15th of each month)')
    ax.set_ylabel('Time of Day')
    ax.set_title("Sun Calculations for " + str(ty) + ", " + str(round(latitude,4)) + " lat and " + str(round(longitude,4)) + " long")
    ax.yaxis_date()
    ax.yaxis.set_major_formatter(mpl.dates.DateFormatter('%H:%M'))
    ax.set_ylim(0, 1)
    ax.plot(months_axis, np.array(sunset_times)/86400, c='orange')
    ax.plot(months_axis, np.array(sunrise_times)/86400, c='gold')
    colour_list = ["#00004d", "#02bae3", "#c4f4ff", "#ffffff", "#ffff00", "#ffd000"]
    cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", colour_list)
    month_letters = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']

    # plot glare colouring
    x_points = []
    y_points = []
    colours = []
    step_size = len(glare_volumes[0])
    # for each day
    for i in range(0, len(glare_volumes), 1):
        # plot all glare points
        colours.extend(glare_volumes[i])
        x_points.extend([i+1]*step_size)
        y_points.extend(np.linspace(0, 86400-step_size/24, step_size).astype(int)/86400)

    sc = ax.scatter(x_points, y_points, c=colours, cmap=cmap, alpha=0.6, marker = '_', s=10*144, vmin=0, vmax=0.05)
    
    # replot all points with 0 glare volume as a black
    ind = np.where(np.array(colours) == 0.0)
    ax.scatter(np.array(x_points)[ind], np.array(y_points)[ind], c='#000000', alpha=0.6, marker = '_', s=10*144, zorder=1)
    
    cbar = fig.colorbar(sc, fraction=0.046, pad=0.04)
    cbar.set_label("Glare Percentage (%)")     

    # save the plot
    fig.savefig(filename + "_monthly.png", bbox_inches='tight', dpi=1200)

else:
    # produce a csv file with times
    open(filename + ".csv", "wb")
    with open(filename + ".csv", "a", newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Day", "Sunrise", "Sunset", "Glare Index"])
        for i in range(0, len(days_axis), 1):
            sunrise_time = str(timedelta(seconds=sunrise_times[i]))
            sunset_time = str(timedelta(seconds=sunset_times[i]))
            row = [days_axis[i], sunrise_time, sunset_time]
            row.extend(list(glare_volumes[i]))
            writer.writerow(row)

    # plot sunsets and sunrises and glare times
    fig=plt.figure()
    ax=fig.add_subplot(111)
    ax.set_xlabel('Day of Year')
    ax.set_ylabel('Time of Day')
    ax.yaxis_date()
    ax.yaxis.set_major_formatter(mpl.dates.DateFormatter('%H:%M'))
    ax.set_ylim(0, 1)
    ax.plot(days_axis, np.array(sunset_times)/86400, c='orange')
    ax.plot(days_axis, np.array(sunrise_times)/86400, c='gold')
    colour_list = ["#00004d", "#02bae3", "#c4f4ff", "#ffffff", "#ffff00", "#ffd000"]
    cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", colour_list)
    month_letters = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']
    # plot gridlines for months
    for i in range(0, len(td_las_list), 1):
        month_day = sum(td_las_list[:i+1])
        month_half = td_las_list[i]/2
        if i < 11:
            ax.plot([month_day, month_day], np.array([0, 1]), c='gray')
        ax.text(month_day - month_half - 4, 80000/86400, month_letters[i], fontsize=12, c='white')

    # plot glare colouring
    x_points = []
    y_points = []
    colours = []
    step_size = len(glare_volumes[0])
    # for each day
    for i in range(0, len(glare_volumes), 1):
        # plot all glare points
        colours.extend(glare_volumes[i])
        x_points.extend([i+1]*step_size)
        y_points.extend(np.linspace(0, 86400-step_size/24, step_size).astype(int)/86400)

    sc = ax.scatter(x_points, y_points, c=colours, cmap=cmap, alpha=0.6, marker = 'o', s=10*(144/step_size), vmin=0, vmax=0.05)

    # replot all points with 0 glare volume as a black
    ind = np.where(np.array(colours) == 0.0)
    ax.scatter(np.array(x_points)[ind], np.array(y_points)[ind], c='#000000', alpha=0.6, marker = 'o', s=10*(144/step_size), zorder=1)
    cbar = fig.colorbar(sc, fraction=0.046, pad=0.04)
    cbar.set_label("Glare Percentage (%)")

    if ver_flag == True:    
        # for verification points, find the intersection over union in time ranges
        IoU_avg = []
        no_inter = 0
        for i in range(0, len(glare_start_check_seconds), 1):
            # display verification range as list of epochs
            ver_list = np.linspace(glare_start_check_seconds[i], glare_finish_check_seconds[i], int((glare_finish_check_seconds[i] - glare_start_check_seconds[i])/60) + 1)
            
            # display glare values as list of epochs
            alg_list = np.where(glare_volumes[glare_check_days[i]-1]>0)[0]*60

            ver_or_alg = len(np.intersect1d(ver_list, alg_list))
            ver_and_alg = (len(ver_list) - ver_or_alg) + (len(alg_list) - ver_or_alg) + ver_or_alg

            if ver_or_alg/ver_and_alg <= 0:
                no_inter = no_inter + 1
            else:
                IoU_avg.append(ver_or_alg/ver_and_alg)

        IoU_avg = np.mean(IoU_avg)
        ax.scatter(glare_check_days, np.array(glare_start_check_seconds)/86400, color='white', zorder=1)  
        ax.scatter(glare_check_days, np.array(glare_finish_check_seconds)/86400, color='darkslategray', zorder=1)

        # save the plot
        fig.savefig(filename + "_IoU_" + str(round(IoU_avg, 2)) + "_no_inters_" + str(no_inter) + ".png", bbox_inches='tight', dpi=1200)
    else:
        # save the plot
        fig.savefig(filename + ".png", bbox_inches='tight', dpi=1200)

print("done")

Run the below cell to generate glare images for phone images

In [ ]:
folder_name = "phone_images/"

IoU_list = []
water_cover_list = []

# for every filename in the folder
for filename_i in os.listdir(folder_name):
    # skip the masks
    if filename_i[-4:] == '.png':
        continue    
    
    filename = "Verification_image_for_" + filename_i + "_dist_type_" + th_dist_type + "_schlick_" + str(schlick_flag)
    
    # extract parameters
    params = filename_i.split("_")
    ty = int(params[14][4:8])
    tm = int(params[14][2:4])
    td = int(params[14][0:2])
    th = int(params[4][0:2])
    tmin = int(params[4][2:4])
    ts = 0
    cam_focal_length = float(params[13].replace('p','.').split("m")[0])
    cam_sensor_width = float(params[10].replace('p','.'))
    cam_sensor_height = float(params[11].replace('p','.'))
    if params[8][-1] == 'S':
        latitude = -1*float(params[8].replace('p','.')[:-1])   # -ve is south
    else:
        latitude = float(params[8].replace('p','.')[:-1])
    if params[9][-1] == 'W':
        longitude = -1*float(params[9].replace('p','.')[:-1])   # -ve is west
    else:
        longitude = float(params[9].replace('p','.')[:-1])
    
    th_cam_h = float(params[6].replace('p','.')[:-3])
    if params[7][0] == 'n':
        th_cam_v = -1*float(params[7].replace('p','.')[1:-3])
    else:
        th_cam_v = float(params[7].replace('p','.')[0:-3])
    
    if params[5][0] == 'p':
        utc_time_add = float(params[5][1:].replace('p','.'))
    else:
        utc_time_add = -1*float(params[5][1:].replace('p','.'))

    # Algorithm input parameters setup
    cam_view_horiz = 2*math.degrees(math.atan(cam_sensor_width/(2*cam_focal_length)))  # 17 # angle range for view
    cam_view_vert = 2*math.degrees(math.atan(cam_sensor_height/(2*cam_focal_length))) # 8.54 #  angle range for view 
    th_cam_h_max = (th_cam_h + cam_view_horiz/2)
    th_cam_h_min = (th_cam_h - cam_view_horiz/2)
    th_cam_v_max = th_cam_v + cam_view_vert/2
    th_cam_v_min = th_cam_v - cam_view_vert/2 
    print(filename_i)
    print("Horizontal field of view = " + str(round(cam_view_horiz, 2)))
    print("Vertical field of view = " + str(round(cam_view_vert,2)))
    print("Depression angle = " + str(round(th_cam_v,2)))
    print("Bearing angle = " + str(round(th_cam_h,2)))
    
    """############ 3: Extract θ_zen,ϕ_az from Astropy using known latitude, longitude, time and timezone ##############"""
    #https://stackoverflow.com/questions/47663082/how-can-we-compute-solar-position-at-a-given-place-on-a-given-day-and-time
    loc = coord.EarthLocation(lon = longitude*u.deg, lat=latitude*u.deg)
    time_val = datetime(ty, tm, td, th, tmin, ts)
    input_time = Time(str(time_val - timedelta(hours = utc_time_add)).replace(" ", "T"), format='isot', scale='utc') # UTC time
    altaz = coord.AltAz(location=loc, obstime=input_time)
    sun = coord.get_sun(input_time)

    # convert azimuth and zenith to degrees
    az_str = str(sun.transform_to(altaz).az)
    th_az = float(az_str.split('d')[0]) + float(az_str.split('d')[1].split('m')[0])/60 \
                  + float(az_str.split('d')[1].split('m')[1].split('s')[0])/3600
    zen_str = str(sun.transform_to(altaz).zen)
    th_zen = float(zen_str.split('d')[0]) + float(zen_str.split('d')[1].split('m')[0])/60 \
                  + float(zen_str.split('d')[1].split('m')[1].split('s')[0])/3600
    
    """############ 4: If daytime (θ_zen ≤ 90): ##############"""
    # if the sun is not up, assume there is no glare and skip to next iteration
    if th_zen > 90:
        print("not daytime, something must be wrong with input parameters")
        break
            
    """############ 5: Calc. ϕ_(ref,h) and θ_(ref,v) for pairs of θ_wx,θ_wy in matrix (Eqs. 5 and 6, Eqs. 9 and 10 for θ_zen < 2θ_wx) ###########"""
    th_sv = 90 - th_zen

    # Schlicks's approximation
    if schlick_flag == True:
        # calculate angle of incidence between sunray and surface normal
        th_wx_rd = np.deg2rad(th_wx)
        th_wy_rd = np.deg2rad(th_wy)
        th_zen_rd = np.deg2rad(th_zen)
        th_az_rd = np.deg2rad(0) # this is always oriented in the direction of the ray of light (always 0 for the way the geometry is setup)
        u_vec = np.array([np.tan(th_wx_rd), np.tan(th_wy_rd), 1])
        v_vec = np.array([np.sin(th_zen_rd)*np.cos(th_az_rd), np.sin(th_zen_rd)*np.sin(th_az_rd), np.cos(th_zen_rd)])
        th_i_num = u_vec.dot(v_vec)
        th_i_den = np.sqrt(u_vec.dot(u_vec))*np.sqrt(v_vec.dot(v_vec))
        th_i = np.rad2deg(np.arccos(th_i_num/th_i_den))
        np.where(np.isnan(th_i), 0, th_i)
        r_coef = 0.0209 + 0.9791*(1 - np.cos(np.deg2rad(th_i)))**5

    # handle out of bounds angles
    th_wx_1 = np.where(th_zen < 2*th_wx, th_wx, np.NAN)
    th_wy_1 = np.where(th_zen < 2*th_wx, th_wy, np.NAN)
    th_wx_2 = np.where(th_zen >= 2*th_wx, th_wx, np.NAN)
    th_wy_2 = np.where(th_zen >= 2*th_wx, th_wy, np.NAN)

    # Equation 8,10 for special flip direction case
    num = np.multiply(np.tan(np.deg2rad(180 - (th_sv + 2*th_wx_1))), np.cos(np.deg2rad(2*th_wy_1)))
    den = np.sqrt(1 + np.multiply(np.square(np.tan(np.deg2rad(180 - (th_sv + 2*th_wx_1)))), np.square(np.sin(np.deg2rad(2*th_wy_1)))))
    th_wv = np.rad2deg(np.arctan(np.divide(num, den)))
    th_ref_v_1 = th_wv

    # Equation 7,9 for special flip direction case
    th_az_dev = -1*np.rad2deg(np.arctan(np.multiply(np.tan(np.deg2rad(180 - (th_sv + 2*th_wx_1))), np.sin(np.deg2rad(2*th_wy_1)))))
    th_ref_h_1 = ((th_az + 180) % 360) + th_az_dev

    # Equation 4,6 for standard case
    num = np.multiply(np.tan(np.deg2rad(th_sv + 2*th_wx_2)), np.cos(np.deg2rad(2*th_wy_2)))
    den = np.sqrt(1 + np.multiply(np.square(np.tan(np.deg2rad(th_sv + 2*th_wx_2))), np.square(np.sin(np.deg2rad(2*th_wy_2)))))
    th_wv = np.rad2deg(np.arctan(np.divide(num, den)))
    th_ref_v_2 = th_wv

    # Equation 3,5 for standard case
    th_az_dev = np.rad2deg(np.arctan(np.multiply(np.tan(np.deg2rad(th_sv + 2*th_wx_2)), np.sin(np.deg2rad(2*th_wy_2)))))
    th_ref_h_2 = th_az + th_az_dev

    # combine matrices again with correct indexing
    th_ref_v_1[np.isnan(th_ref_v_1)] = 0
    th_ref_h_1[np.isnan(th_ref_h_1)] = 0
    th_ref_v_2[np.isnan(th_ref_v_2)] = 0
    th_ref_h_2[np.isnan(th_ref_h_2)] = 0
    th_ref_v = th_ref_v_1 + th_ref_v_2
    th_ref_h = th_ref_h_1 + th_ref_h_2           

    """############ 6: For every ≈ 2.1°×2.1° cell in the FOV: ############"""
    # setup the range of cells at about 2.1 degrees per cell
    v_inc = (th_cam_v_max - th_cam_v_min)/2.1
    h_inc = (th_cam_h_max - th_cam_h_min)/2.1
    th_cam_vs = np.linspace(th_cam_v_min, th_cam_v_max, int(v_inc))
    th_cam_hs = np.linspace(th_cam_h_min, th_cam_h_max, int(h_inc))
    all_deps = th_ref_v.flatten(order='C') 
    all_azs = th_ref_h.flatten(order='C') 
    glare_img = np.ones([len(th_cam_vs), len(th_cam_hs)]) # **********
    glare_percs_i = []
    for m in range(0, len(th_cam_vs), 1):
        for n in range(0, len(th_cam_hs), 1):  
            """############ 7: Set Φ_(ref,h) values to 1 if within range of ϕ_(cam,h), set Θ_(ref,v) values to 1 if within range of θ_(cam,v) #############"""
            # set to 0 all th_wx and th_wy that don't reflect into the camera fov
            th_cam_v_min_m = th_cam_vs[m] - 0.5*(th_cam_v_max - th_cam_v_min)/(int(v_inc))
            th_cam_v_max_m = th_cam_vs[m] + 0.5*(th_cam_v_max - th_cam_v_min)/(int(v_inc))
            th_cam_h_min_n = th_cam_hs[n] - 0.5*(th_cam_h_max - th_cam_h_min)/(int(h_inc))
            th_cam_h_max_n = th_cam_hs[n] + 0.5*(th_cam_h_max - th_cam_h_min)/(int(h_inc))

            # if the camera has any FOV above the horizon, don't include any negative th_ref_v angles, 
            # rather there should be 0 reflection since there is no water here
            if th_cam_v_max_m < 0: 
                glare_img[m][n] = 0 # *************
                continue

            # work out glare percentage
            deps_glare_1 = np.where(all_deps >= th_cam_v_min_m, 1, 0)
            deps_glare_2 = np.where(all_deps <= th_cam_v_max_m, 1, 0)
            deps_glare = deps_glare_1 & deps_glare_2
            azs_glare_1 = np.where(all_azs >= th_cam_h_min_n, 1, 0)
            azs_glare_2 = np.where(all_azs <= th_cam_h_max_n, 1, 0)
            azs_glare = azs_glare_1 & azs_glare_2
            
            """############ 8: Θ_all=(Φ_(ref,h)  AND Θ_(ref,v) )*Θ_dist*Θ_R(θ_i ) ##########"""
            # apply roughness distribution
            th_all = np.multiply(deps_glare & azs_glare, th_dist_f)

            # apply Schlicks's approximation
            if schlick_flag == True:
                th_all = np.multiply(th_all, r_coef.flatten(order='C'))
            
            """############ 9: glare_percentage_i = (∑Θ_all)/(∑Θ_dist )*100 ##########"""
            # get volume under roughness distribution 
            #- since width and length of each cell is the same for both distributions, 
            # they cancel when used for a ratio. Therefore only need to sum the bin heights.
            vol_dist = np.sum(th_dist_f)
            
            # get volume under output distribution
            vol_output = np.sum(th_all)
            
            # find percentage of total volume
            glare_perc = (vol_output/vol_dist)*100

            # add to glare percentage average and the image
            glare_percs_i.append(glare_perc)
            glare_img[m][n] = glare_perc     # ***********
    
    """############ 10: glare percentage = mean(glare_percentage_i’s) ############"""
    glare_perc_average = np.mean(glare_percs_i)

    # create and show image     ***********
    print(str(time_val))
    inp_img = cv2.imread(folder_name + filename_i) # *********
    inp_img_height, inp_img_width, _ = inp_img.shape # *********
    glare_img = cv2.resize(glare_img, (0,0), fx=20, fy=20, interpolation = cv2.INTER_NEAREST)*255
    glare_out_img = cv2.resize(glare_img, (inp_img_width,inp_img_height))
    glare_out_img = np.where(glare_out_img > 255, 255, glare_out_img)
    glare_out_img = glare_out_img.astype('uint8')
    colour_img = cv2.cvtColor(glare_out_img, cv2.COLOR_GRAY2RGB)
    colour_img[np.all(colour_img == (0, 0, 0), axis=-1)] = (50,0,0)
    
    # download mask
    man_mask = cv2.imread(folder_name + filename_i[:-4] + "_mask.png")   
    man_mask = cv2.cvtColor(man_mask, cv2.COLOR_RGB2GRAY)
    
    # mask both with where man_mask is white
    non_water_mask = np.where(man_mask == 255, 0, 1)
    
    # do IoU check
    alg_mask = colour_img[:,:,1]
    alg_mask = np.where(alg_mask > 0, 255, 0)
    alg_mask = np.multiply(non_water_mask, alg_mask)
    man_mask = np.where(man_mask > 0, 1, 0)
    man_mask = np.multiply(non_water_mask, man_mask)
    man_mask = np.where(man_mask > 0, 255, 0)
    mask_and = np.logical_and(alg_mask, man_mask)
    mask_and = np.where(mask_and > 0, 255, 0)
    mask_or = np.logical_or(alg_mask, man_mask)
    mask_or = np.where(mask_or > 0, 255, 0)
    IoU = np.sum(np.logical_and(alg_mask, man_mask))/np.sum(np.logical_or(alg_mask, man_mask))
    water_cover = (2*min(np.sum(alg_mask),np.sum(man_mask)))/(np.sum(alg_mask) + np.sum(man_mask))
    print("Water coverage similarity = " + str(round(water_cover, 2)))
    print("Intersection over Union = " + str(round(IoU,2)))
    water_cover_list.append(water_cover)
    IoU_list.append(IoU)
    
    # produce a transparent overaly image of the mask on the original image for visualization purposes.
    #check_img = cv2.addWeighted(inp_img, 0.7, colour_img, 0.3, 0)
    
    cv2.imwrite(filename_i + "_dist_type_" + th_dist_type + "_schlick_" + str(schlick_flag) + "_glare_perc_" + str(round(glare_perc_average,3)) + "_IoU_" + str(round(IoU,2)) + ".png", colour_img)
    
    # debugging images
    """
    cv2.imshow('alg', alg_mask.astype('uint8'))
    cv2.imshow('man', man_mask.astype('uint8'))
    cv2.imshow('and', mask_and.astype('uint8'))
    cv2.imshow('or', mask_or.astype('uint8'))    
    cv2.namedWindow('gl', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('gl', 1280, 720)
    cv2.imshow('gl', glare_out_img.astype('uint8'))
    cv2.namedWindow('glare', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('glare', 1280, 720)
    cv2.imshow('glare', check_img)
    cv2.waitKey()
    cv2.destroyAllWindows()
    """

print("Average Water Coverage Similarity = " + str(round(np.mean(water_cover_list),2)))    
print("Average Intersection over Union = " + str(round(np.mean(IoU_list),2)))
print("done")